In [1]:
# Import necessary libraries

from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # For OpenAI support
from google.adk.tools import ToolContext

# Convenience libraries for working with Neo4j inside of Google ADK
from neo4j_for_adk import graphdb, tool_success, tool_error

import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.CRITICAL)

print("Libraries imported.")

Libraries imported.


In [2]:
%pip install neo4j-for-adk --quiet
%pip install neo4j --quiet

ERROR: Could not find a version that satisfies the requirement neo4j-for-adk (from versions: none)
ERROR: No matching distribution found for neo4j-for-adk
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Convenience libraries for working with Neo4j inside of Google ADK
from neo4j_for_adk import graphdb

In [4]:
# Comprehensive Neo4j Diagnostics
import os
import socket
import time
from neo4j import GraphDatabase

print("=== Neo4j Connection Diagnostics ===\n")

# 1. Check environment variables
print("1. Environment Variables:")
print(f"   NEO4J_URI: {os.getenv('NEO4J_URI', 'Not set')}")
print(f"   NEO4J_USERNAME: {os.getenv('NEO4J_USERNAME', 'Not set')}")
print(f"   NEO4J_PASSWORD: {'*' * len(os.getenv('NEO4J_PASSWORD', '')) if os.getenv('NEO4J_PASSWORD') else 'Not set'}")
print(f"   NEO4J_DATABASE: {os.getenv('NEO4J_DATABASE', 'Not set')}\n")

# 2. Check if ports are open
print("2. Port Connectivity Test:")
def check_port(host, port):
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(5)
        result = sock.connect_ex((host, port))
        sock.close()
        return result == 0
    except:
        return False

ports_to_check = [
    ('127.0.0.1', 7687, 'Neo4j Bolt (IPv4)'),
    ('localhost', 7687, 'Neo4j Bolt (localhost)'),
    ('127.0.0.1', 7474, 'Neo4j Browser (IPv4)'),
    ('localhost', 7474, 'Neo4j Browser (localhost)')
]

for host, port, desc in ports_to_check:
    is_open = check_port(host, port)
    status = "✅ OPEN" if is_open else "❌ CLOSED"
    print(f"   {desc} ({host}:{port}): {status}")

print()

# 3. Try different connection approaches
print("3. Connection Attempts:")

# Try with explicit IPv4
try:
    driver = GraphDatabase.driver("bolt://127.0.0.1:7687", auth=("neo4j", "password123"))
    with driver.session() as session:
        result = session.run("RETURN 'IPv4 Direct' as test, timestamp() as time")
        record = result.single()
        print(f"   ✅ IPv4 Direct: {record['test']} at {record['time']}")
    driver.close()
except Exception as e:
    print(f"   ❌ IPv4 Direct: {type(e).__name__}: {str(e)[:100]}...")

# Try with localhost
try:
    driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password123"))
    with driver.session() as session:
        result = session.run("RETURN 'localhost' as test, timestamp() as time")
        record = result.single()
        print(f"   ✅ localhost: {record['test']} at {record['time']}")
    driver.close()
except Exception as e:
    print(f"   ❌ localhost: {type(e).__name__}: {str(e)[:100]}...")

# Try with environment variables
try:
    uri = os.getenv('NEO4J_URI', 'bolt://127.0.0.1:7687')
    username = os.getenv('NEO4J_USERNAME', 'neo4j')
    password = os.getenv('NEO4J_PASSWORD', 'password123')
    driver = GraphDatabase.driver(uri, auth=(username, password))
    with driver.session() as session:
        result = session.run("RETURN 'Env Vars' as test, timestamp() as time")
        record = result.single()
        print(f"   ✅ Environment Vars: {record['test']} at {record['time']}")
    driver.close()
except Exception as e:
    print(f"   ❌ Environment Vars: {type(e).__name__}: {str(e)[:100]}...")

print("\n=== Recommendations ===")
print("If all connection attempts fail:")
print("1. Wait 30-60 seconds after container start for Neo4j to fully initialize")
print("2. Check Docker logs: docker logs neo4j")
print("3. Verify container is running: docker ps | grep neo4j")
print("4. Try recreating container if issues persist:")
print("   docker rm -f neo4j")
print("   docker run -d --name neo4j -p 7474:7474 -p 7687:7687 -e NEO4J_AUTH=neo4j/password123 neo4j:latest")

=== Neo4j Connection Diagnostics ===

1. Environment Variables:
   NEO4J_URI: bolt://127.0.0.1:7687
   NEO4J_USERNAME: neo4j
   NEO4J_PASSWORD: ***********
   NEO4J_DATABASE: neo4j

2. Port Connectivity Test:
   Neo4j Bolt (IPv4) (127.0.0.1:7687): ✅ OPEN
   Neo4j Bolt (localhost) (localhost:7687): ✅ OPEN
   Neo4j Browser (IPv4) (127.0.0.1:7474): ✅ OPEN
   Neo4j Browser (localhost) (localhost:7474): ✅ OPEN

3. Connection Attempts:
   ✅ IPv4 Direct: IPv4 Direct at 1768127922923
   ✅ localhost: localhost at 1768127922935
   ✅ Environment Vars: Env Vars at 1768127922945

=== Recommendations ===
If all connection attempts fail:
1. Wait 30-60 seconds after container start for Neo4j to fully initialize
2. Check Docker logs: docker logs neo4j
3. Verify container is running: docker ps | grep neo4j
4. Try recreating container if issues persist:
   docker rm -f neo4j
   docker run -d --name neo4j -p 7474:7474 -p 7687:7687 -e NEO4J_AUTH=neo4j/password123 neo4j:latest


In [5]:
# Check connection to Neo4j by sending a query

neo4j_is_ready = graphdb.send_query("RETURN 'Neo4j is Ready!' as message")

print(neo4j_is_ready)

{'status': 'success', 'query_result': [{'message': 'Neo4j is Ready!'}]}


In [6]:
ner_agent_role_and_goal = """
You are a top-tier algorithm designer for analyzing text files and proposing
the kind of named entities that could be extracted which would be relevant 
for a user's goal.
"""

In [7]:
ner_agent_hints = """
Entities are people, places, things and qualities, but not quantities. 
Your goal is to propose a list of the type of entities, not the actual instances
of entities.

There are two general approaches to identifying types of entities:
- well-known entities: these closely correlate with approved node labels in an existing graph schema
- discovered entities: these may not exist in the graph schema, but appear consistently in the source text

Design rules for well-known entities:
- always use existing well-known entity types. For example, if there is a well-known type "Person", and people appear in the text, then propose "Person" as the type of entity.
- prefer reusing existing entity types rather than creating new ones

Design rules for discovered entities:
- discovered entities are consistently mentioned in the text and are highly relevant to the user's goal
- always look for entities that would provide more depth or breadth to the existing graph
- for example, if the user goal is to represent social communities and the graph has "Person" nodes, look through the text to discover entities that are relevant like "Hobby" or "Event"
- avoid quantitative types that may be better represented as a property on an existing entity or relationship.
- for example, do not propose "Age" as a type of entity. That is better represented as an additional property "age" on a "Person".
"""

In [8]:
ner_agent_chain_of_thought_directions = """
Prepare for the task:
- use the 'get_user_goal' tool to get the user goal
- use the 'get_approved_files' tool to get the list of approved files
- use the 'get_well_known_types' tool to get the approved node labels

Think step by step:
1. Sample some of the files using the 'sample_file' tool to understand the content
2. Consider what well-known entities are mentioned in the text
3. Discover entities that are frequently mentioned in the text that support the user's goal
4. Use the 'set_proposed_entities' tool to save the list of well-known and discovered entity types
5. Use the 'get_proposed_entities' tool to retrieve the proposed entities and present them to the user for their approval
6. If the user approves, use the 'approve_proposed_entities' tool to finalize the entity types
7. If the user does not approve, consider their feedback and iterate on the proposal
"""

In [9]:
ner_agent_instruction = f"""
{ner_agent_role_and_goal}
{ner_agent_hints}
{ner_agent_chain_of_thought_directions}
"""

#print(ner_agent_instruction)

In [10]:
# tools to propose and approve entity types
PROPOSED_ENTITIES = "proposed_entity_types"
APPROVED_ENTITIES = "approved_entity_types"

def set_proposed_entities(proposed_entity_types: list[str], tool_context:ToolContext) -> dict:
    """Sets the list proposed entity types to extract from unstructured text."""
    tool_context.state[PROPOSED_ENTITIES] = proposed_entity_types
    return tool_success(PROPOSED_ENTITIES, proposed_entity_types)

def get_proposed_entities(tool_context:ToolContext) -> dict:
    """Gets the list of proposed entity types to extract from unstructured text."""
    return tool_context.state.get(PROPOSED_ENTITIES, [])

def approve_proposed_entities(tool_context:ToolContext) -> dict:
    """Upon approval from user, records the proposed entity types as an approved list of entity types 

    Only call this tool if the user has explicitly approved the suggested files.
    """
    if PROPOSED_ENTITIES not in tool_context.state:
        return tool_error("No proposed entity types to approve. Please set proposed entities first, ask for user approval, then call this tool.")
    tool_context.state[APPROVED_ENTITIES] = tool_context.state.get(PROPOSED_ENTITIES)
    return tool_success(APPROVED_ENTITIES, tool_context.state[APPROVED_ENTITIES])

def get_approved_entities(tool_context:ToolContext) -> dict:
    """Get the approved list of entity types to extract from unstructured text."""
    return tool_context.state.get(APPROVED_ENTITIES, [])


In [11]:
def get_well_known_types(tool_context:ToolContext) -> dict:
    """Gets the approved labels that represent well-known entity types in the graph schema."""
    construction_plan = tool_context.state.get("approved_construction_plan", {})
    # approved labels are the keys for each construction plan entry where `construction_type` is "node"
    approved_labels = {entry["label"] for entry in construction_plan.values() if entry["construction_type"] == "node"}
    return tool_success("approved_labels", approved_labels)



In [12]:
from tools import get_approved_user_goal, get_approved_files, sample_file
ner_agent_tools = [
    get_approved_user_goal, get_approved_files, sample_file,
    get_well_known_types,
    set_proposed_entities,
    approve_proposed_entities
]

In [13]:
file_result = sample_file(file_path="product_reviews/gothenburg_table_reviews.md",tool_context=None)

print(file_result["content"])

# Gothenburg Table Reviews

Scraped from https://www.between2furns.com/Gothenburg-Table/dp/B0BQJWJWJW

## Rating: ★★★★★ (5/5)
This table is the centerpiece of our dining room! The modern design is stunning yet timeless, and the quality is exceptional. We've had it for 6 months now and it still looks brand new despite daily use with two kids. Assembly took about an hour with two people. The surface is easy to clean and surprisingly resistant to scratches. Highly recommend!

- @akollegger (Cambridge)

---

## Rating: ★★★☆☆ (3/5)
The Gothenburg Table looks beautiful, I'll give it that. But assembly was a nightmare - some of the pre-drilled holes didn't line up properly and I had to drill new ones. Once assembled, it's sturdy enough, but I expected better quality control for this price point. The table top also shows fingerprints very easily which is annoying for daily use.

- @furniture_lover92 (Seattle)

---

## Rating: ★★★★☆ (4/5)
love love LOVE the look of this table!! perfect size for